In [1]:
# retail_ordering_system.py
# Run with: python retail_ordering_system.py
# Requires Python 3.6+

from math import floor

try:
    from prettytable import PrettyTable
    HAVE_PRETTYTABLE = True
except Exception:
    HAVE_PRETTYTABLE = False


def format_table(items):
    """Return a simple table string if PrettyTable isn't available."""
    if HAVE_PRETTYTABLE:
        t = PrettyTable()
        t.field_names = ["Name", "Price (USD)", "Quantity"]
        for it in items:
            t.add_row([it["Name"], f"{it['Price']:.2f}", it["Quantity"]])
        return str(t)
    else:
        # fallback plain table
        lines = []
        header = f"{'Name':30} | {'Price (USD)':12} | {'Quantity':8}"
        sep = "-" * len(header)
        lines.append(header)
        lines.append(sep)
        for it in items:
            lines.append(f"{it['Name']:30} | {it['Price']:12.2f} | {it['Quantity']:8}")
        return "\n".join(lines)


def find_product_by_name(products, name):
    """Case-insensitive lookup, returns product dict or None."""
    name_lower = name.strip().lower()
    for p in products:
        if p["Name"].lower() == name_lower:
            return p
    return None


def get_positive_int(prompt):
    """Prompt until the user provides a positive integer."""
    while True:
        val = input(prompt).strip()
        if val.isdigit():
            n = int(val)
            if n > 0:
                return n
        print("Please enter a positive integer.")


def stationary_store():
    """
    Stationary store interaction.
    Returns total_stationary_price (in USD) and updates stationary inventory in place.
    Loyalty discount: every 50 units purchased grants +2% loyalty discount.
    The loyalty percent accumulates across purchases.
    """
    stationary_products = [
        {"Name": "Notebook", "Price": 3.50, "Quantity": 200},
        {"Name": "Pen (Blue)", "Price": 0.75, "Quantity": 1000},
        {"Name": "Pen (Black)", "Price": 0.75, "Quantity": 1000},
        {"Name": "Highlighter", "Price": 1.25, "Quantity": 300},
        {"Name": "Marker", "Price": 2.00, "Quantity": 150},
        {"Name": "Stapler", "Price": 8.50, "Quantity": 40},
    ]

    print("\n--- Stationary Store ---")
    print(format_table(stationary_products))

    total_stationary_price = 0.0
    loyalty_percent = 0.0  # accumulates across purchases

    while True:
        name = input("\nEnter stationary product name (or 'done' to finish stationary shopping): ").strip()
        if name.lower() == "done":
            break
        product = find_product_by_name(stationary_products, name)
        if not product:
            print("Product not found in stationary store. Please try again (case-insensitive).")
            continue

        print(f"Available quantity for {product['Name']}: {product['Quantity']}")
        qty = get_positive_int(f"Enter quantity of {product['Name']} to buy: ")

        if qty > product["Quantity"]:
            print(f"Only {product['Quantity']} units available. Please enter a lower quantity.")
            continue

        # Determine loyalty increment for this purchase
        loyalty_increment = floor(qty / 50) * 2.0  # 2% per 50 units
        loyalty_percent += loyalty_increment

        # Apply the accumulated loyalty discount to this purchase
        discount_pct = loyalty_percent
        discounted_price = qty * product["Price"] * (1 - discount_pct / 100.0)

        # Update inventory and totals
        product["Quantity"] -= qty
        total_stationary_price += discounted_price

        print(f"Added {qty} x {product['Name']} at unit price ${product['Price']:.2f}")
        print(f"Loyalty increment this purchase: {loyalty_increment:.2f}% -> cumulative loyalty: {loyalty_percent:.2f}%")
        print(f"Discounted subtotal for this item: ${discounted_price:.2f}")

        # Option to continue or stop stationary shopping
        cont = input("Buy another stationary item? (y/n): ").strip().lower()
        if cont != "y":
            break

    print(f"Total stationary purchases (USD): ${total_stationary_price:.2f}")
    return total_stationary_price


def main():
    # Part 1 - General products inventory
    products = [
        {"Name": "Milk (1L)", "Price": 1.20, "Quantity": 500},
        {"Name": "Cheese (200g)", "Price": 2.50, "Quantity": 300},
        {"Name": "Yogurt (500g)", "Price": 1.80, "Quantity": 400},
        {"Name": "Butter (250g)", "Price": 3.20, "Quantity": 150},
        {"Name": "Ice Cream (500ml)", "Price": 4.50, "Quantity": 200},
    ]

    print("Welcome to the Retail Ordering System (USD).")
    total_discounted_price = 0.0

    while True:
        print("\nAvailable products:")
        print(format_table(products))

        name = input("\nEnter product name to buy (or 'done' to finish): ").strip()
        if name.lower() == "done":
            break

        product = find_product_by_name(products, name)
        if not product:
            print("Product not found. Please try again (case-insensitive match).")
            continue

        print(f"Available quantity for {product['Name']}: {product['Quantity']}")
        qty = get_positive_int(f"Enter quantity of {product['Name']} to buy: ")

        if qty > product["Quantity"]:
            print(f"Insufficient stock. Only {product['Quantity']} units available. Please enter a lower quantity.")
            continue

        # Discount: 5% per 250 units, capped at 25%
        discount_steps = floor(qty / 250)
        discount_pct = discount_steps * 5.0
        if discount_pct > 25.0:
            discount_pct = 25.0

        discounted_price = qty * product["Price"] * (1 - discount_pct / 100.0)

        # Update inventory and running total
        product["Quantity"] -= qty
        total_discounted_price += discounted_price

        print(f"Added {qty} x {product['Name']} at ${product['Price']:.2f} each.")
        print(f"Volume discount: {discount_pct:.2f}% -> discounted line total: ${discounted_price:.2f}")

        cont = input("Add another product? (y/n): ").strip().lower()
        if cont != "y":
            break

    # Part 2 - Stationary store
    use_stationary = input("\nWould you like to visit the stationary store? (y/n): ").strip().lower()
    total_stationary_price = 0.0
    if use_stationary == "y":
        total_stationary_price = stationary_store()

    subtotal_usd = total_discounted_price + total_stationary_price
    print(f"\nSubtotal (USD) before delivery/pick-up: ${subtotal_usd:.2f}")

    # Part 3 - Delivery vs Pick-up
    delivery_choice = input("Choose 'delivery' or 'pick-up' (case-insensitive). Any other input = no extra charge: ").strip().lower()
    extra_charge = 0.0
    if delivery_choice == "delivery":
        extra_charge = 200.0
        print("Delivery charge applied: $200.00")
    elif delivery_choice == "pick-up" or delivery_choice == "pickup":
        extra_charge = 50.0
        print("Pick-Up charge applied: $50.00")
    else:
        print("Invalid or no option chosen for delivery/pick-up. No extra charge applied.")

    total_usd = subtotal_usd + extra_charge
    print(f"Total (USD) after extra charges: ${total_usd:.2f}")

    # Currency conversion
    currency = input("Select payment currency (USD, EUR, EGP). Default USD if invalid: ").strip().upper()
    rates = {"USD": 1.0, "EUR": 0.92, "EGP": 30.90}
    if currency not in rates:
        print("Invalid currency selected. Defaulting to USD.")
        currency = "USD"

    converted_total = total_usd * rates[currency]

    # Formatting: two decimals for USD/EUR, 2 decimals for EGP too (you can change)
    print("\n--- Order Summary ---")
    print(f"Products total (USD): ${total_discounted_price:.2f}")
    print(f"Stationary total (USD): ${total_stationary_price:.2f}")
    print(f"Extra charge (USD): ${extra_charge:.2f}")
    print(f"Final total in {currency}: {converted_total:.2f} {currency}")

    print("\nYour order is on the way. Thank you for shopping with us!")


if __name__ == "__main__":
    main()


Welcome to the Retail Ordering System (USD).

Available products:
Name                           | Price (USD)  | Quantity
--------------------------------------------------------
Milk (1L)                      |         1.20 |      500
Cheese (200g)                  |         2.50 |      300
Yogurt (500g)                  |         1.80 |      400
Butter (250g)                  |         3.20 |      150
Ice Cream (500ml)              |         4.50 |      200
Available quantity for Yogurt (500g): 400
Added 250 x Yogurt (500g) at $1.80 each.
Volume discount: 5.00% -> discounted line total: $427.50

Subtotal (USD) before delivery/pick-up: $427.50
Invalid or no option chosen for delivery/pick-up. No extra charge applied.
Total (USD) after extra charges: $427.50
Invalid currency selected. Defaulting to USD.

--- Order Summary ---
Products total (USD): $427.50
Stationary total (USD): $0.00
Extra charge (USD): $0.00
Final total in USD: 427.50 USD

Your order is on the way. Thank you for sho